In [121]:
import pandas as pd

In [156]:
suppliers = pd.read_csv("suppliers.csv")
suppliers["T_ITEM"] = suppliers["T_ITEM"].str.strip()
suppliers_names = pd.read_csv("suppliers_names.csv")

suppliers = suppliers.merge(suppliers_names, how="left", right_on="suplier_id", left_on="T_OTBP")
suppliers["T_OTBP"] = suppliers["T_OTBP"].str.strip()


In [177]:
suppliers_from_sourcing = pd.read_excel("supps_from_sourcing.xlsx", 
                                        usecols=['SupplierCode', 'HyperionCode','PurchaseOrga',
                                                'OriginERPName','OriginERPCode'],
                                       dtype={'OriginERPCode':str, "OriginERPName":str})
suppliers_from_sourcing
mask01 = suppliers_from_sourcing["HyperionCode"] == "KPOL"
mask02 = suppliers_from_sourcing["OriginERPName"] == "240"
suppliers_from_sourcing = suppliers_from_sourcing[mask01]
suppliers_from_sourcing

,SupplierCode,HyperionCode,PurchaseOrga,OriginERPName,OriginERPCode
941,SUP000140,KPOL,KPOL,205,100254
942,SUP000140,KPOL,KPOL,240,100254
1044,SUP000157,KPOL,KPOL,205,316164
1045,SUP000157,KPOL,KPOL,240,316164
1046,SUP000157,KPOL,KPOL,ln,316164
...,...,...,...,...,...
218232,SUP193835,KPOL,KPOL,NaN,NaN
218265,SUP193907,KPOL,KPOL,NaN,NaN
218268,SUP193912,KPOL,KPOL,240,SUP193912
218284,SUP193982,KPOL,KPOL,NaN,NaN


In [174]:
file9 = pd.ExcelWriter("supps.xlsx")
suppliers_from_sourcing.to_excel(file9, index=False)
file9.close()

In [179]:
suppliers_from_sourcing_test = suppliers_from_sourcing.copy()
mask03 = suppliers_from_sourcing_test["OriginERPCode"] == "110286"
suppliers_from_sourcing_test = suppliers_from_sourcing_test[mask03]
suppliers_from_sourcing_test

,SupplierCode,HyperionCode,PurchaseOrga,OriginERPName,OriginERPCode
70206,SUP023449,KPOL,KPOL,205,110286
70207,SUP023449,KPOL,KPOL,240,110286


In [161]:
suppliers_merged = suppliers.merge(suppliers_from_sourcing, how="left", left_on="suplier_id", right_on="OriginERPCode")
suppliers_merged

,T_ITEM,T_KITM,T_OTBP,suplier_id,supplier_name,SupplierCode,HyperionCode,PurchaseOrga,OriginERPName,OriginERPCode
0,64154018,1,110286,110286,PMC Hydraulika Sp. Zo.o.,NaN,NaN,NaN,NaN,NaN
1,LNOB_JP110849,1,110347,110347,ZUWA-Zumpe GmbH,NaN,NaN,NaN,NaN,NaN
2,1718010327,1,100506,100506,ZF CV Distribution Sweden AB,NaN,NaN,NaN,NaN,NaN
3,90144,1,111581,111581,ENERPOL s.c. B.B.K Żabkiewicz,NaN,NaN,NaN,NaN,NaN
4,59505995,1,100415,100415,TITRY ry,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
136429,459-8497,1,SUP182582,SUP182582,Malmar Group BV,NaN,NaN,NaN,NaN,NaN
136430,459-8498,1,SUP182582,SUP182582,Malmar Group BV,NaN,NaN,NaN,NaN,NaN
136431,459-8499,1,SUP182582,SUP182582,Malmar Group BV,NaN,NaN,NaN,NaN,NaN
136432,459-8500,1,SUP182582,SUP182582,Malmar Group BV,NaN,NaN,NaN,NaN,NaN


# loading item purchase

In [125]:
item_purchase = pd.read_csv("item_purchase_240.csv", usecols=["T_ITEM", "item_puchase_price", "currency"])
item_purchase["T_ITEM"] = item_purchase["T_ITEM"].str.strip()
currency_rates = pd.read_excel("currency_rates.xlsx")
item_purchase_with_currency = item_purchase.merge(currency_rates, how="left", on="currency")
item_purchase_with_currency["price_in_eur"] = item_purchase_with_currency["item_puchase_price"] / item_purchase_with_currency["rate"]
item_purchase_with_currency.drop(columns=["currency", "item_puchase_price", "rate"], inplace=True)
test1 = item_purchase_with_currency["T_ITEM"] == "A60683.0100"
item_purchase_with_currency[test1]

,T_ITEM,price_in_eur


# loading michine list

In [126]:
products = pd.read_excel("items_division.xlsx")
mask = products["Business Unit"] == 240
products = products[mask]
products["Machine"] = products["Machine"].str.strip()

# loading bom table

In [127]:
boms = pd.read_csv("all_boms_240.csv")
boms["main_item"] = boms["main_item"].str.strip()
boms["subitem"] = boms["subitem"].str.strip()
mask12 = ~boms["subitem"].isin(products["Machine"])
mask15 = ~boms["main_item"].isin(item_purchase["T_ITEM"])

subs_without_price = boms.copy()
test4 = ~subs_without_price["subitem"].isin(item_purchase_with_currency["T_ITEM"])
test5 = ~subs_without_price["subitem"].isin(subs_without_price["main_item"])
subs_without_price = subs_without_price[test4 & test5]
mask16 = ~boms["subitem"].isin(subs_without_price["subitem"])

items_zero_quant = boms.copy()
test6 = items_zero_quant["quantity"] == 0
items_zero_quant = items_zero_quant[test6]
mask17 = ~boms["subitem"].isin(items_zero_quant["subitem"])

weird_item = boms.copy()
test7 = weird_item["subitem"] == "-"
weird_item = weird_item[test7]
mask18 = ~boms["subitem"].isin(weird_item["subitem"])

boms = boms[mask12 & mask15 & mask16 & mask17 & mask18]
boms_with_price = boms.merge(item_purchase_with_currency, how="left", left_on="subitem", right_on="T_ITEM")
boms_with_price.drop(columns="T_ITEM", inplace=True)
test3 = boms_with_price["subitem"] == "A25469.9999"
boms_with_price[test3]


,main_item,subitem,quantity,price_in_eur


# getting a df with main items only

In [128]:
mask6 = ~boms_with_price["main_item"].isin(boms_with_price["subitem"])
main_items_only = boms_with_price.copy()
main_items_only = main_items_only[mask6]
mask9 = main_items_only["main_item"].isin(products["Machine"])
main_items_only = main_items_only[mask9]
main_items_only

,main_item,subitem,quantity,price_in_eur
4526,C00100.6350,A62419.1200,-1.0,NaN
8912,C00100.6351,A69248.2100,-1.0,NaN
14347,C00100.6050,A60645.0200,-1.0,NaN
15655,Z99711,922010.0022,12.0,0.546352
15814,Z99752V,922297.0184,12.0,13069.870000
...,...,...,...,...
6038325,Z99690V,A63254.0100,1.0,NaN
6040916,C00260.0170,A55286.2600,1.0,NaN
6046498,Z99765,A59992.0100,1.0,2.430000
6047048,Z99765,A71343.0200,1.0,NaN


# getting a df with main items and sub in main

In [129]:
main_items_and_sub_in_main = main_items_only.merge(boms_with_price, how="left", left_on="subitem",
                                                   right_on="main_item")
main_items_and_sub_in_main.reset_index(inplace=True, drop=True)
main_items_and_sub_in_main.drop("main_item_y", inplace=True, axis=1)
print(main_items_and_sub_in_main.columns)
main_items_and_sub_in_main.columns = ["machine", "level1", "level1_q", "level1_price", "level2", "level2_q", "level2_price"]
main_items_and_sub_in_main.insert(loc = 4, column="level1_price_calc", value=main_items_and_sub_in_main["level1_q"] * main_items_and_sub_in_main["level1_price"])

main_items_and_sub_in_main.insert(loc = 8, column="level2_q_calc", value=main_items_and_sub_in_main["level1_q"] * main_items_and_sub_in_main["level2_q"])
main_items_and_sub_in_main["level2_price_calc"] = main_items_and_sub_in_main["level2_q_calc"] * main_items_and_sub_in_main["level2_price"]
main_items_and_sub_in_main

Index(['main_item_x', 'subitem_x', 'quantity_x', 'price_in_eur_x', 'subitem_y',
       'quantity_y', 'price_in_eur_y'],
      dtype='object')


,machine,level1,level1_q,level1_price,level1_price_calc,level2,level2_q,level2_price,level2_q_calc,level2_price_calc
0,C00100.6350,A62419.1200,-1.0,NaN,NaN,923284.0002,12.0,0.820000,-12.0,-9.840000
1,C00100.6350,A62419.1200,-1.0,NaN,NaN,5443,4.0,1.310000,-4.0,-5.240000
2,C00100.6350,A62419.1200,-1.0,NaN,NaN,A40395.0100,4.0,1.239866,-4.0,-4.959464
3,C00100.6350,A62419.1200,-1.0,NaN,NaN,920728.236,4.0,0.380000,-4.0,-1.520000
4,C00100.6350,A62419.1200,-1.0,NaN,NaN,99122,4.0,0.207100,-4.0,-0.828400
...,...,...,...,...,...,...,...,...,...,...
31822,Z99765,A71343.0200,1.0,NaN,NaN,98150,2.0,0.021600,2.0,0.043200
31823,Z99765,A71343.0200,1.0,NaN,NaN,A71213.0200,1.0,354.270000,1.0,354.270000
31824,Z99765,A71343.0200,1.0,NaN,NaN,922564.0006,1.0,0.079200,1.0,0.079200
31825,Z99765,A71343.0200,1.0,NaN,NaN,3090,1.0,0.102000,1.0,0.102000


# changing column names

In [130]:

test_cols = ['machine', 'level1', 'level1_price', 'level1_q', 'level1_price_calc',
       'level2', 'level2_q', 'level2_price', 'level2_q_calc',
       'level2_price_calc']
main_items_and_sub_in_main = main_items_and_sub_in_main[test_cols]
main_items_and_sub_in_main

,machine,level1,level1_price,level1_q,level1_price_calc,level2,level2_q,level2_price,level2_q_calc,level2_price_calc
0,C00100.6350,A62419.1200,NaN,-1.0,NaN,923284.0002,12.0,0.820000,-12.0,-9.840000
1,C00100.6350,A62419.1200,NaN,-1.0,NaN,5443,4.0,1.310000,-4.0,-5.240000
2,C00100.6350,A62419.1200,NaN,-1.0,NaN,A40395.0100,4.0,1.239866,-4.0,-4.959464
3,C00100.6350,A62419.1200,NaN,-1.0,NaN,920728.236,4.0,0.380000,-4.0,-1.520000
4,C00100.6350,A62419.1200,NaN,-1.0,NaN,99122,4.0,0.207100,-4.0,-0.828400
...,...,...,...,...,...,...,...,...,...,...
31822,Z99765,A71343.0200,NaN,1.0,NaN,98150,2.0,0.021600,2.0,0.043200
31823,Z99765,A71343.0200,NaN,1.0,NaN,A71213.0200,1.0,354.270000,1.0,354.270000
31824,Z99765,A71343.0200,NaN,1.0,NaN,922564.0006,1.0,0.079200,1.0,0.079200
31825,Z99765,A71343.0200,NaN,1.0,NaN,3090,1.0,0.102000,1.0,0.102000


# Getting all levels

In [131]:
level2 = main_items_and_sub_in_main.copy()
last_level = False
level = 2
while last_level == False:

    x = "level"
    z = "_q"
    p = "_price"
    y = str(level)
    column_name_lvl = x+y
    mask1 = level2[column_name_lvl].isin(boms["main_item"])
    mask2 = level2[column_name_lvl].isin(item_purchase["T_ITEM"])
    test = level2.copy()
    test2 = level2.copy()
    test = test[mask1]
    test2 = test2[mask2]


    if test.empty:
        last_level = True

    else:
        y = str(level)
        column_name_lvl = x + y
        column_name_q = x+y+z
        column_name_p = x+y+p
        level2 = level2.merge(boms_with_price, how="left", left_on=column_name_lvl, right_on="main_item")
        level += 1
        y = str(level)
        t = str(level-1)
        calc = "_calc"
        column_name_lvl = x + y
        column_name_q = x+y+z
        column_name_p = x + y + p
        column_name_q_t = x+t+z+calc
        

        level2.drop("main_item", inplace=True, axis=1)
        level2.rename(columns={"subitem": column_name_lvl, "quantity": column_name_q, "price_in_eur":column_name_p}, inplace=True)
        level2[column_name_q+calc] = level2[column_name_q_t] * level2[column_name_q]
        level2[column_name_p+calc] = level2[column_name_q+calc] * level2[column_name_p]
        level2.drop(columns=[column_name_q], inplace=True)

level2.rename(columns={"level1_q":"level1_q_calc"},inplace=True)
level2.drop(columns=["level2_q"], inplace=True)
level2.columns

Index(['machine', 'level1', 'level1_price', 'level1_q_calc',
       'level1_price_calc', 'level2', 'level2_price', 'level2_q_calc',
       'level2_price_calc', 'level3', 'level3_price', 'level3_q_calc',
       'level3_price_calc', 'level4', 'level4_price', 'level4_q_calc',
       'level4_price_calc', 'level5', 'level5_price', 'level5_q_calc',
       'level5_price_calc', 'level6', 'level6_price', 'level6_q_calc',
       'level6_price_calc'],
      dtype='object')

# Grouping by machines

In [132]:
machines = level2.groupby(by="machine")
machines_names = machines.groups.keys()


# Getting final boms

In [133]:
level_list = []
list_of_boms = []
levels = ()
for machine in machines_names:
    
    list_of_dfs = []
    dict = {}
    col_num = 0
    cost_item = machines.get_group(machine)
    cost_item1 = cost_item.copy()
    #cost_item1.dropna(how="all", axis=1, inplace=True)
    print(machine)
    for x in cost_item1.columns:
        level_num = x[5]
        dict[col_num] = level_num
        col_num +=1
        
    levels = set(dict.values())
    levels = sorted(levels)
    levels.remove("n")
    
    
    for level in levels:
        list_of_cols = []
        for key, value in dict.items():
            if value == level:
                list_of_cols.append(key)
        list_of_cols.insert(0,0)
        #print(list_of_cols)
        df = cost_item1.iloc[:,list_of_cols].copy()
        #print(df.columns)
        df.columns = ['machine', 'item', 'price', 'quant',
           'price*quant']
        list_of_dfs.append(df)
        list_of_dfs
    bom_concat = pd.concat(list_of_dfs)
    list_of_boms.append(bom_concat)
print(list_of_boms)
boms_ready = pd.concat(list_of_boms)
boms_ready.dropna(subset='price*quant', inplace=True)

C00100.2705
C00100.2726
C00100.6001
C00100.6004
C00100.6013
C00100.6040
C00100.6050
C00100.6140
C00100.6141
C00100.6142
C00100.6143
C00100.6201
C00100.6206
C00100.6209
C00100.6350
C00100.6351
C00100.6410
C00100.6413
C00100.6704
C00260.0170
Z99104
Z99690V
Z99691V
Z99711
Z99715
Z99721
Z99730
Z99752V
Z99765
Z99811
Z99842
Z99848
Z99911
[           machine         item  price  quant  price*quant
224    C00100.2705  921797.0017  112.2    2.0        224.4
415    C00100.2705  A60140.1100    NaN    1.0          NaN
1253   C00100.2705  A65464.0100    NaN    1.0          NaN
1254   C00100.2705  A65464.0100    NaN    1.0          NaN
1255   C00100.2705  A65464.0100    NaN    1.0          NaN
...            ...          ...    ...    ...          ...
46693  C00100.2705          NaN    NaN    NaN          NaN
46701  C00100.2705          NaN    NaN    NaN          NaN
46702  C00100.2705          NaN    NaN    NaN          NaN
46703  C00100.2705          NaN    NaN    NaN          NaN
46704  C00100.27

# Merging with suppliers

In [180]:
boms_suppliers = boms_ready.merge(suppliers, how="left", left_on="item", right_on="T_ITEM")

boms_suppliers = boms_suppliers.merge(suppliers_from_sourcing, how="left", 
                                     left_on="suplier_id", right_on="OriginERPCode")
boms_suppliers

,machine,item,price,quant,price*quant,T_ITEM,T_KITM,T_OTBP,suplier_id,supplier_name,SupplierCode,HyperionCode,PurchaseOrga,OriginERPName,OriginERPCode
0,C00100.2705,921797.0017,112.200000,2.0,224.400000,921797.0017,1,110202,110202,Exide Technologies S.A.,SUP023376,KPOL,KPOL,205,110202
1,C00100.2705,921797.0017,112.200000,2.0,224.400000,921797.0017,1,110202,110202,Exide Technologies S.A.,SUP023376,KPOL,KPOL,240,110202
2,C00100.2705,921797.0017,112.200000,2.0,224.400000,921797.0017,1,110202,110202,Exide Technologies S.A.,SUP023376,KPOL,KPOL,ln,110202
3,C00100.2705,A64876.0100,10263.190000,1.0,10263.190000,A64876.0100,1,104209,104209,Fortaco Sp. z.o.o.,SUP023120,KPOL,KPOL,205,104209
4,C00100.2705,A64876.0100,10263.190000,1.0,10263.190000,A64876.0100,1,104209,104209,Fortaco Sp. z.o.o.,SUP023120,KPOL,KPOL,240,104209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161084,Z99911,920043.225,0.386400,8.0,3.091200,920043.225,1,318814,318814,Bufab Poland Sp. z o. o,SUP020313,KPOL,KPOL,205,318814
161085,Z99911,920043.225,0.386400,8.0,3.091200,920043.225,1,318814,318814,Bufab Poland Sp. z o. o,SUP020313,KPOL,KPOL,240,318814
161086,Z99911,A69220.0100,8.653642,2.0,17.307284,A69220.0100,1,319631,319631,GIPO Sp. z o.o.,SUP023517,KPOL,KPOL,240,319631
161087,Z99911,920043.225,0.386400,8.0,3.091200,920043.225,1,318814,318814,Bufab Poland Sp. z o. o,SUP020313,KPOL,KPOL,205,318814


# Creating the excel file

In [181]:
file3 = pd.ExcelWriter("bom_final.xlsx")
boms_suppliers.to_excel(file3, index=False)
file3.close()